In [170]:
import pandas as pd
dict_df = pd.read_excel("Monthly_retail_sales_workbook.xlsx", skiprows= 4, sheet_name = None)

Get the information on the produced dictionary

In [171]:
#currently reads in all sheets to a dictionary of dataframes without names
# print(type(dict_df))
# print(len(dict_df))
# for key in dict_df:
#     print(key)

Create a single dataframe from dictionary to work with as an example
This cleans the top and bottom of the dataframe of unwanted data

In [172]:
df = dict_df.get('1992')
df = df.drop(['TOTAL'], axis = 1)
df = df.drop([0])
df.rename(columns = {'Unnamed: 0':'NAPCSCode', 'Unnamed: 1':'NAPCSname'}, inplace = True)
print(type(df['NAPCSCode']))
drop_rows = df[df['NAPCSCode'] == 722].index.tolist()[1]
df = df.iloc[:drop_rows-1] #This drops the bottom rows that are not needed if i can locate the second index

<class 'pandas.core.series.Series'>


This splits the single dataframe into two separate dataframes for adjusted and non adjusted values.
It also removed the last of unwanted data that includes combination values and additional top bottom values
I tried removing rows that had a comma in them based on NAPCSCode, but that crashed the kernel and was far more time consuming then necessary.

In [173]:
# adjusted_indexlist = df[df['NAPCSname'] == 'ADJUSTED(2)'].index.tolist()[0]
# adjusted_df = df.iloc[adjusted_indexlist+7:,:]
# adjusted_df = adjusted_df.drop([75, 77])
# non_adjusteddf = df.iloc[:adjusted_indexlist -6,:]
# number_of_rows = 7
# non_adjusteddf = non_adjusteddf.iloc[number_of_rows: , :]
# non_adjusteddf = non_adjusteddf.drop([9, 14])
# #at this point the dataframes are split and done.  They need writing to new dictionaries
# display(non_adjusteddf)
# display(adjusted_df)

One thing to note is that that the dataframes need to be pivoted for the ml algorithm and that the code will likely need to be removed for that step as well or at least moved.

This should be the full automation to run through the entire dictionary clean and write the dataframes into new dictionaries.  It is the combination of the above codes int a for loop

keys are nonadjusted_year or adjusted_year

In [174]:
adjusted_dict = {}
nonadjusted_dict = {}
for key in dict_df:
    if key == '2022':
        print('Skipping 2022 due to incomplete data')
    else:
        df = dict_df.get('{}'.format(key))
        df = df.drop(['TOTAL'], axis = 1)
        df = df.drop([0])
        df.rename(columns = {'Unnamed: 0':'NAPCSCode', 'Unnamed: 1':'NAPCSname'}, inplace = True)
        drop_rows = df[df['NAPCSCode'] == 722].index.tolist()[1] #chooses the seoond occurence of 722
        df = df.iloc[:drop_rows-1]
        # code for bottom section adjusted dataframe
        adjusted_indexlist = df[df['NAPCSname'] == 'ADJUSTED(2)'].index.tolist()[0]
        adjusted_df = df.iloc[adjusted_indexlist+7:,:]
        adjusted_df = adjusted_df.drop([75, 77])
        #code for top non adjusted dataframe
        non_adjusteddf = df.iloc[:adjusted_indexlist -6,:]
        number_of_rows = 7
        non_adjusteddf = non_adjusteddf.iloc[number_of_rows: , :]
        non_adjusteddf = non_adjusteddf.drop([9, 14])
        #Write dataframes to new dictionaries
        adjusted_dict.update({'adjusted_{}'.format(key) : adjusted_df})
        nonadjusted_dict.update({'nonadjusted_{}'.format(key): non_adjusteddf})
# for key in adjusted_dict:
#     print(key)
    
# for key in nonadjusted_dict:
#     print(key)

Skipping 2022 due to incomplete data


The below code takes out the header and splits it into a list of month and year.  
It then converts the month into the integer representation.
It needs to be looped or added to the loop based on what we need.
The 'Total''s were left to represent the unmodified version of the table.

In [198]:
from datetime import datetime
dict_df2 = pd.read_excel("Monthly_retail_sales_workbook.xlsx", header = None, skiprows= 4, sheet_name = None)
df2 = dict_df2.get('1992')
# display(df2)
daterowsplit = df2.iloc[0]
# print(daterowsplit)
for i in daterowsplit:
    if type(i) == str:
        if i.find('.') != -1:
            x = i.split('. ')
        else:
            x = i.split(' ')
    if x[0] != 'TOTAL':
        x[0] = datetime.strptime(x[0], '%b').month
#     print(x)
# display(df)

['TOTAL']
['TOTAL']
[1, '1992']
[2, '1992']
[3, '1992']
[4, '1992']
[5, '1992']
[6, '1992']
[7, '1992']
[8, '1992']
[9, '1992']
[10, '1992']
[11, '1992']
[12, '1992']
['TOTAL']


,NAPCSCode,NAPCSname,Jan. 1992,Feb. 1992,Mar. 1992,Apr. 1992,May 1992,Jun. 1992,Jul. 1992,Aug. 1992,Sep. 1992,Oct. 1992,Nov. 1992,Dec. 1992
1,NaN,"Retail and food services sales, total",146376,147079,159336,163669,170068,168663,169890,170364,164617,173655,171547,208838
2,NaN,Retail sales and food services excl motor vehi...,116565,115862,124200,127587,133608,130274,132076,134928,128734,136917,139187,175771
3,NaN,Retail sales and food services excl gasoline s...,134277,135499,147064,151226,156619,155236,156016,156539,151477,159992,158483,195350
4,NaN,Retail sales and food services excl motor vehi...,104466,104282,111928,115144,120159,116847,118202,121103,115594,123254,126123,162283
5,NaN,"Retail sales, total",130683,131244,142488,147175,152420,151849,152586,152476,148158,155987,154824,191347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,45299,All other gen. merchandise stores,2682,2621,2493,2541,2499,2475,2479,2523,2504,2562,2576,2666
100,453,Miscellaneous stores retailers,4318,4308,4292,4391,4356,4357,4468,4494,4549,4809,5059,5059
101,454,Nonstore retailers,6515,6110,6018,6327,6550,6653,6891,6766,6714,6665,6610,6597
102,4541,Electronic shopping and mail order houses,2668,2632,2477,2757,2874,3065,3238,3137,3064,3062,3029,3032


This uses the produces separated dictionary of dataframes from above and pivots them for the machine learning algorithm.

In [197]:
df4 = nonadjusted_dict['nonadjusted_2021']
# df4 = df4.stack()
# df4 = df4.pivot(columns = ['NAPCSCode','NAPCSname'])
print(type(df4))
display(df4)

<class 'pandas.core.frame.DataFrame'>


,NAPCSCode,NAPCSname,Jan. 2021,Feb. 2021,Mar. 2021,Apr. 2021,May 2021,Jun. 2021,Jul. 2021,Aug. 2021,Sep. 2021,Oct. 2021,Nov. 2021,Dec. 2021
8,441,Motor vehicle and parts dealers,100629,99324,141659,138891,137600,132297,129123,123620,118654,120186,115837,122304
10,4411,Automobile dealers,87117,85259,120463,117514,116807,111154,109169,104362,100594,101704,98587,105140
11,44111,New car dealers,77391,74874,104050,101980,102538,96934,95559,90256,86553,87437,85574,92521
12,44112,Used car dealers,9726,10385,16413,15534,14269,14220,13610,14106,14041,14267,13013,12619
13,4413,"Automotive parts, acc., and tire stores",7686,7489,10168,9838,9420,9673,9704,9829,9432,9844,9437,9565
15,442,Furniture and home furnishings stores,10432,9916,12401,11951,11925,11730,12027,12091,11974,12040,12795,12678
16,4421,Furniture stores,6009,5657,7074,6770,6742,6406,6589,6542,6614,6629,6656,6455
17,4422,Home furnishings stores,(S),(S),(S),(S),(S),(S),(S),(S),(S),(S),(S),(S)
18,44221,Floor covering stores,(S),(S),(S),(S),(S),(S),(S),(S),(S),(S),(S),(S)
19,442299,All other home furnishings stores,(S),(S),(S),(S),(S),(S),(S),(S),(S),(S),(S),(S)
